# **Tarea 2**

## _Punto 1:_

### 1. Modelado del problema como un MDP

Modele este problema como un MDP. Detalle todos los elementos del MDP:

- **Estados**  
- **Recompensas**  
- **Acciones**  
- **Dinámica de transición**:  
  $$ p(s', r \mid s, a) \quad \forall s, s', r, a $$  
- **Factor de descuento**:  
  $$ \gamma $$

In [58]:
import numpy as np

Se definen las razones de probabilidad del dado cargado, lo que significa que las probailidades no son uniformes, algunas tienen más probabilidad de salir que otras. En lugar de un dado justo con una probabilidad uniforme de 1/6, aquí se asignan valores específicos: sacar un 1 o 6 tiene una probabilidad de 10%, un 2 o un 5 ocurre el 15% del tiempo, y un 3 o un 4 sucede con mayor frecuencia (20%). Esto afecta las transiciones del MDP, ya que los valores medios (3 y 4) serán más comunes, influyendo en la estrategia óptima del jugador al moverse en el tablero.

In [71]:
probabilidades = np.array([0.1, 0.2, 0.3, 0.2, 0.1, 0.1])  # dado cargado

A continuación, se definen dos diccionarios en Python que modelan las escaleras y serpientes del tablero, donde las claves representan las casillas de inicio y los valores indican las casillas destino. El diccionario escaleras indica las casillas donde un jugador avanza automáticamente a una posición más alta, mientras que serpientes representa las casillas que obligan al jugador a retroceder, dificultando su progreso. Estas transiciones no lineales afectan la dinámica del juego y se incorporan en la función de probabilidad de transición del MDP, asegurando que el modelo refleje correctamente las reglas del juego.

In [72]:
# Definir posiciones de escaleras y serpientes
escaleras = {8: 26, 21: 82, 43: 77, 50: 91, 54: 93, 66: 87, 62: 96, 80: 100} # modelado de 8 escaleras
serpientes = {52: 11, 69: 33, 92: 51, 48: 9, 73: 1, 55: 7, 46: 5, 95: 24, 64: 36, 44: 22, 98: 28, 83: 19, 59: 17} # modelado de serpientes

Se definen mediante listas, los estados correspondientes a victoria o derrota denotados en la grafica de color azul y rojo respectivamente

In [73]:
# Casillas de ganar y perder
estadoGanar = [80, 100]  # Casillas azules
estadoPerder = [23, 37, 45, 67, 89]  # Casillas rojas

Se modela la variable gamma que en el MDP permite dar relevancia a la evolución de los estados en el tiempo, el valor de 0.9 se parametriza para dar mayor relevancia a los estados recientes en comparación a los estados más antiguos

In [74]:
# Factor de descuento
gamma = 0.9

Se modela una función que modela las probabbilidades de transición en el talero de juego, esta calcula la distribución de probabilidad de transición en el tablero de escaleras y serpientes, modelando cómo un jugador se moverá desde un estado actual 𝑠 tras tomar una acción 𝑎 y lanzar el dado. Para cada posible resultado del dado, ajusta el estado futuro sumando o restando el valor obtenido según la acción tomada. Luego, maneja los rebotes en los extremos del tablero si el jugador supera la casilla 100 o cae por debajo de la casilla 1. Posteriormente, aplica las reglas de escaleras y serpientes, trasladando al jugador si cae en una de estas casillas. Finalmente, almacena la probabilidad acumulada de alcanzar cada estado en un vector, asegurando que la suma total de probabilidades sea 1, y lo retorna, permitiendo definir la matriz de transición 𝑃(𝑠′∣𝑠,𝑎) clave en el MDP.

In [92]:
def calcular_nuevo_estado(s, a, resultado_dado):
    # Verificar si el estado actual es terminal
    if s in estadoGanar or s in estadoPerder:
        return s  # No se mueve si ya está en un estado terminal

    # Calcular el nuevo estado según la acción
    if a == "avanzar":
        nuevo_s = s + resultado_dado
    elif a == "retroceder":
        nuevo_s = s - resultado_dado
    else:
        raise ValueError("Acción no válida")

    # Rebote en los extremos (solo si no es un estado terminal)
    if nuevo_s > 100:
        nuevo_s = 100 - (nuevo_s - 100)  # Rebote si se supera 100
    elif nuevo_s < 1:
        nuevo_s = 100 - (1 - nuevo_s)  # Rebote si se retrocede más allá de 1

    # Verificar si el nuevo estado es terminal
    if nuevo_s in estadoGanar or nuevo_s in estadoPerder:
        return nuevo_s  # No se aplican escaleras o serpientes en estados terminales

    # Escaleras
    if nuevo_s in escaleras:
        nuevo_s = escaleras[nuevo_s]

    # Serpientes
    if nuevo_s in serpientes:
        nuevo_s = serpientes[nuevo_s]

    return nuevo_s

Se modela la función de recomensas, esta asigna una recompensa a cada estado 𝑠 según su impacto en el juego, retornando +1 si el jugador alcanza una casilla de victoria (estadoGanar), -1 si cae en una casilla de derrota (estadoPerder), y -0.01 en cualquier otro caso como penalización por movimiento, incentivando así estrategias que minimicen el número de pasos y maximicen la recompensa total en el MDP.

In [93]:
def obtener_recompensa(s):
    if s in estadoGanar:
        return 100
    elif s in estadoPerder:
        return -100
    else:
        return -1


In [94]:
# Función para simular un paso en el MDP
def paso_mdp(s, a):
    # Lanzar el dado
    resultado_dado = np.random.choice([1, 2, 3, 4, 5, 6], p=probabilidades)

    # Calcular nuevo estado
    nuevo_s = calcular_nuevo_estado(s, a, resultado_dado)

    # Obtener recompensa
    r = obtener_recompensa(nuevo_s)

    # Verificar si el estado es terminal
    terminal = (nuevo_s in estadoGanar) or (nuevo_s in estadoPerder)

    return nuevo_s, r, terminal

In [101]:
# Ejemplo de simulación
def simular_juego():
    s = 1  # Estado inicial
    historial = []
    terminal = False

    while not terminal:
        a = "avanzar" # Elegir acción (en este ejemplo, siempre avanzar)
        s, r, terminal = paso_mdp(s, a) # Tomar un paso en el MDP       
        historial.append((s, r)) # Guardar historial
        print(f"Estado: {s}, Recompensa: {r}")

    print("Fin del juego.")
    return historial

# Ejecutar simulación
historial = simular_juego()

Estado: 7, Recompensa: -1
Estado: 9, Recompensa: -1
Estado: 14, Recompensa: -1
Estado: 16, Recompensa: -1
Estado: 19, Recompensa: -1
Estado: 24, Recompensa: -1
Estado: 27, Recompensa: -1
Estado: 30, Recompensa: -1
Estado: 34, Recompensa: -1
Estado: 40, Recompensa: -1
Estado: 77, Recompensa: -1
Estado: 19, Recompensa: -1
Estado: 25, Recompensa: -1
Estado: 29, Recompensa: -1
Estado: 33, Recompensa: -1
Estado: 39, Recompensa: -1
Estado: 22, Recompensa: -1
Estado: 28, Recompensa: -1
Estado: 31, Recompensa: -1
Estado: 34, Recompensa: -1
Estado: 36, Recompensa: -1
Estado: 40, Recompensa: -1
Estado: 22, Recompensa: -1
Estado: 25, Recompensa: -1
Estado: 27, Recompensa: -1
Estado: 33, Recompensa: -1
Estado: 39, Recompensa: -1
Estado: 41, Recompensa: -1
Estado: 22, Recompensa: -1
Estado: 28, Recompensa: -1
Estado: 31, Recompensa: -1
Estado: 34, Recompensa: -1
Estado: 40, Recompensa: -1
Estado: 5, Recompensa: -1
Estado: 9, Recompensa: -1
Estado: 11, Recompensa: -1
Estado: 15, Recompensa: -1
Estad

##### Veificación del MDP:

Se realizan verificaciones para corroborar que el MDP está correctamente estructurado

Se verifica que cada estado tiene una distribución de probabilidad válida mediante la suma de probabilidades de transición.

In [96]:
# Verificar que no hay estados repetidos o no válidos
estados = set(range(1, 101))  # Estados del 1 al 100
estados_terminales = set(estadoGanar + estadoPerder)

# Verificar que los estados terminales están dentro del rango
assert all(s in estados for s in estados_terminales), "Estados terminales no válidos"

# Verificar que no hay solapamientos entre estados de ganar y perder
assert len(set(estadoGanar).intersection(set(estadoPerder))) == 0, "Solapamiento entre estados de ganar y perder"

In [97]:
# Verificar que las acciones son válidas
acciones_validas = {"avanzar", "retroceder"}
acciones = {"avanzar", "retroceder"}

assert acciones.issubset(acciones_validas), "Acciones no válidas"

In [98]:
# Verificar recompensas en estados terminales
for s in estadoGanar:
    assert obtener_recompensa(s) == 100, f"Recompensa incorrecta en estado ganar {s}"

for s in estadoPerder:
    assert obtener_recompensa(s) == -100, f"Recompensa incorrecta en estado perder {s}"

# Verificar recompensas en estados no terminales
for s in range(1, 101):
    if s not in estadoGanar and s not in estadoPerder:
        assert obtener_recompensa(s) == -1, f"Recompensa incorrecta en estado no terminal {s}"

In [99]:
# Verificar que al avanzar desde 99 con un dado de 1, el jugador llega a 100 (estado terminal)
assert calcular_nuevo_estado(99, "avanzar", 1) == 100, "Error al llegar a 100"

# Verificar rebote al retroceder más allá de 1
assert calcular_nuevo_estado(2, "retroceder", 2) == 100, "Error en rebote al retroceder más allá de 1"

# Verificar que no hay rebote en estados terminales
assert calcular_nuevo_estado(100, "avanzar", 2) == 100, "Error: Rebote en estado terminal 100"
assert calcular_nuevo_estado(80, "avanzar", 2) == 80, "Error: Rebote en estado terminal 80"
assert calcular_nuevo_estado(23, "avanzar", 2) == 23, "Error: Rebote en estado terminal 23"

# Verificar escaleras
assert calcular_nuevo_estado(8, "avanzar", 1) == 26, "Error en escalera en casilla 8"
assert calcular_nuevo_estado(21, "avanzar", 1) == 82, "Error en escalera en casilla 21"

# Verificar serpientes
assert calcular_nuevo_estado(52, "avanzar", 1) == 11, "Error en serpiente en casilla 52"
assert calcular_nuevo_estado(69, "avanzar", 1) == 33, "Error en serpiente en casilla 69"

# Verificar que no hay bucles infinitos (por ejemplo, escaleras que llevan a serpientes y viceversa)
for s in escaleras:
    assert escaleras[s] not in serpientes, f"Escalera en {s} lleva a una serpiente"

for s in serpientes:
    assert serpientes[s] not in escaleras, f"Serpiente en {s} lleva a una escalera"

AssertionError: Error en rebote al retroceder más allá de 1

In [39]:
assert 0 <= gamma <= 1, "Factor de descuento no válido"

In [40]:
def verificar_transicion(s, a):
    # Calcular todas las transiciones posibles desde el estado s con la acción a
    transiciones = {}
    for resultado_dado in range(1, 7):
        nuevo_s = calcular_nuevo_estado(s, a, resultado_dado)
        r = obtener_recompensa(nuevo_s)
        transiciones[(nuevo_s, r)] = transiciones.get((nuevo_s, r), 0) + prob_dado[resultado_dado - 1]

    # Verificar que la suma de probabilidades es 1
    suma_probabilidades = sum(transiciones.values())
    assert np.isclose(suma_probabilidades, 1), f"Error en la función de transición para s={s}, a={a}"

# Verificar para algunos estados y acciones
verificar_transicion(1, "avanzar")
verificar_transicion(50, "retroceder")
verificar_transicion(99, "avanzar")

NameError: name 'prob_dado' is not defined

In [42]:
def verificar_episodio():
    s = 1  # Estado inicial
    terminal = False
    pasos = 0

    while not terminal:
        a = "avanzar"  # Siempre avanzar para simplificar
        s, r, terminal = paso_mdp(s, a)
        pasos += 1

        # Verificar que no se excede un número razonable de pasos
        assert pasos <= 1000, "El episodio no terminó en un número razonable de pasos"

    # Verificar que el estado final es terminal
    assert s in estadoGanar or s in estadoPerder, "El episodio no terminó en un estado terminal"

# Ejecutar varias simulaciones para verificar
for _ in range(100):
    verificar_episodio()